In [1]:
configPath = "metric_learning/config/cirtorch.yaml"
modelPath = "../../../Models/CIBR/rSfM120k-tl-resnet152-gem-w-f39cada.pth"
imageDatabasePath = "../../../Datasets/wikiart/High_Renaissance"
queryImagePath = "../../../Datasets/custom/15_19_titian_venus-anadyomene.jpg"
folderToStoreResultsPath = "../../../Datasets/custom/RenaissanceCirtorch/"

In [2]:
import os
import pickle
from metric_learning.networks import CirTorch, CirTorchConfig

os.environ['CUDA_VISIBLE_DEVICES'] = "0"

config = CirTorchConfig.create(configPath)
config.defrost()
config.model = modelPath
config.freeze()
cirtorchNetwork = CirTorch(config)
cirtorchNetwork.load()
databasePath = os.path.join(folderToStoreResultsPath, "database.pkl")
if os.path.exists(databasePath):
    cirtorchNetwork.loadDatabase(databasePath)
else:
    databaseImages = [os.path.join(imageDatabasePath, file) for file in os.listdir(imageDatabasePath)]
    cirtorchNetwork.createDatabase(databaseImages)
    cirtorchNetwork.saveDatabase(databasePath)
scores = cirtorchNetwork.query(queryImagePath)

scoresPath = os.path.join(folderToStoreResultsPath, "scores.pkl")
with open(scoresPath, 'wb') as scoresFile:
    pickle.dump(scores, scoresFile)


>> Loading network


d:\UGent\Master Industriele Wetenschappen\Masterproef\Repositories\GANs_Pose_Estimation_on_Art_Collection\.venv_win\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\UGent\Master Industriele Wetenschappen\Masterproef\Repositories\GANs_Pose_Estimation_on_Art_Collection\.venv_win\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "d:\UGent\Master Industriele Wetenschappen\Masterproef\Repositories\GANs_Pose_Estimation_on_Art_Collection\.venv_win\lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "d:\UGent\Master Industriele Wetenschappen\Masterproef\Repositories\GANs_Pose_Estimation_on_Art_Collection\.venv_win\lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "d:\UGent\Master Industriele Wetenschappen\Masterproef\Repositories\GANs_Pose_Estimation_on_Art_Collection\.venv_win\lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "d:\ugent\master industriele wetenschappen\masterproef\repositories\gans_pose_estimation_on_art_collection\lib\c

In [ ]:
import shutil

scoresPath = os.path.join(folderToStoreResultsPath, "scores.pkl")
with open(scoresPath, 'rb') as scoresFile:
    scores = pickle.load(scoresFile)
prevScore = 100
countSameScore = 0
for score in scores:
    if score[1] > 0.15:
        scoreAsInteger = int(score[1]*100)
        if prevScore != scoreAsInteger:
            countSameScore = 1
        else:
            countSameScore += 1
        prevScore = scoreAsInteger
        imagePath = cirtorchNetwork.images[int(score[0])]
        shutil.copy(imagePath, os.path.join(folderToStoreResultsPath, f"{scoreAsInteger}_{countSameScore}_{os.path.split(imagePath)[-1]}"))